**Introduction**

The Pacific Laboratory for Artificial Intelligence (PLAI) is a research group at the University of British Columbia interested in collecting data on how people play and interact in video games. PLAI has publicly released a free Minecraft server, plaicraft.ai, a “free Minecraft in the cloud” to enable their project on generative AI research data collection. However, research efforts and resources are demanding in a project of this size, where the actions of each player are recorded throughout their playthrough. Thus, project lead Frank Wood has offered the undergraduate DSCI100 students an opportunity to assist in answering one of three questions related to the data they have accumulated.

Our group has chosen to address question 2: “We would like to know which "kinds" of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts.” Based on this overarching question, we created our group question of interest: “What experience levels and ages contribute to greater playtime in hours?” Playtime in hours was chosen as the response variable, as we believe it best captures the amount of data players contribute. Thus, predicting higher amounts of playtime from explanatory variables may yield insight into what player characteristics are most significantly associated with larger amounts of data generated. 

To narrow down the overarching inquiry, player age and experience level (Amateur, Beginner, Regular, Pro, Veteran) were chosen as explanatory variables. Age is heavily tied to life stages, influencing factors such as time availability, patterns in cognitive engagement, and gaming habits, which have the potential to affect total playtime. Experience level reflects player commitment and implies dedication towards the game before participation, possibly indicating the amount of time and sessions players are willing to dedicate to gameplay. Thus, we believed these variables would be influential in contributing the most data and subsequent targeted recruitment by narrowing down the scope of age groups and experience levels most likely to have the highest playtime. 

The dataset consists of 196 observations and 9 variables. This includes the columns “experience” (categorical string) representing player Minecraft experience level, “subscribe” (categorical boolean) indicating player subscription to a game-related newsletter, “hashedEmail” (categorical string) as the player’s email, “played_hours” (numerical integer) as the number of played hours, “name” (categorical string), "gender” (categorical string), “age” (numerical integer) in years, and two empty fields “individualID” and “organizationName” (NoneType). Components of the data were sourced from player activity, actions, and interactions on plaicraft.ai, while other aspects such as player names and emails were self-reported. It is not declared whether player age, experience level, and gender were sourced from a self-report measure or player activity.

**Methods and Results**

In [1]:
### Run this cell before continuing.
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsRegressor

# Simplify working with large datasets in Altair
alt.data_transformers.enable('vegafusion')

# Output dataframes instead of arrays
set_config(transform_output="pandas")

In [2]:
url = "https://drive.google.com/uc?export=download&id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz"
players = pd.read_csv(url)
players

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN
...,...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN


In [3]:
url_2 = "https://drive.google.com/uc?export=download&id=14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB"
sessions = pd.read_csv(url_2)
sessions

,hashedEmail,start_time,end_time,original_start_time,original_end_time
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,30/06/2024 18:12,30/06/2024 18:24,1.719770e+12,1.719770e+12
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,17/06/2024 23:33,17/06/2024 23:46,1.718670e+12,1.718670e+12
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,25/07/2024 17:34,25/07/2024 17:57,1.721930e+12,1.721930e+12
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,25/07/2024 03:22,25/07/2024 03:58,1.721880e+12,1.721880e+12
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,25/05/2024 16:01,25/05/2024 16:12,1.716650e+12,1.716650e+12
...,...,...,...,...,...
1530,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,10/05/2024 23:01,10/05/2024 23:07,1.715380e+12,1.715380e+12
1531,7a4686586d290c67179275c7c3dfb4ea02f4d317d9ee0e...,01/07/2024 04:08,01/07/2024 04:19,1.719810e+12,1.719810e+12
1532,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,28/07/2024 15:36,28/07/2024 15:57,1.722180e+12,1.722180e+12
1533,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,25/07/2024 06:15,25/07/2024 06:22,1.721890e+12,1.721890e+12


In [4]:
players_filtered = players[["age", "played_hours", "experience"]]
players_filtered

,age,played_hours,experience
0,9,30.3,Pro
1,17,3.8,Veteran
2,17,0.0,Veteran
3,21,0.7,Amateur
4,21,0.1,Regular
...,...,...,...
191,17,0.0,Amateur
192,22,0.3,Veteran
193,17,0.0,Amateur
194,17,2.3,Amateur


In [5]:
experience_map = {
    "Amateur": 1,
    "Beginner": 2,
    "Regular": 3,
    "Pro": 4,
    "Veteran": 5
}

players_filtered = players_filtered.assign(
    experience_num = players_filtered['experience'].map(experience_map)
)

players_filtered

,age,played_hours,experience,experience_num
0,9,30.3,Pro,4
1,17,3.8,Veteran,5
2,17,0.0,Veteran,5
3,21,0.7,Amateur,1
4,21,0.1,Regular,3
...,...,...,...,...
191,17,0.0,Amateur,1
192,22,0.3,Veteran,5
193,17,0.0,Amateur,1
194,17,2.3,Amateur,1


The data was loaded from a URL to make our code reproducible. To quantify experience level, we assigned each level a number ranging from 1 to 5 in increasing order of experience. Beginner was 1, Amateur was 2, Regular was 3, Pro was 4, and Veteran was 5. 

In [48]:
#split filtered data into training and testing with 70/30 split using random_state = 2000
training_data, testing_data = train_test_split(
    players_filtered, test_size = 0.3, random_state = 2000
)

In [40]:
players_filtered.describe() 

,age,played_hours,experience_num
count,196.000000,196.000000,196.000000
mean,21.280612,5.845918,2.739796
std,9.706346,28.357343,1.568487
min,8.000000,0.000000,1.000000
25%,17.000000,0.000000,1.000000
50%,19.000000,0.100000,2.500000
75%,22.000000,0.600000,4.000000
max,99.000000,223.100000,5.000000


In [41]:
players_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   age               196 non-null    int64   
 1   played_hours      196 non-null    float64 
 2   experience        196 non-null    object  
 3   experience_num    196 non-null    int64   
 4   playtime_grouped  111 non-null    category
dtypes: category(1), float64(1), int64(2), object(1)
memory usage: 6.8+ KB


In [47]:
players_filtered_0 = players_filtered[(players_filtered["played_hours"] <= 160)]
players_filtered_0.describe()
#shows that when outliers removed, mean playtime was 2.7 hours with a std of 13 --> should we make this our filter instead?

,age,played_hours,experience_num
count,193.000000,193.000000,193.000000
mean,21.321244,2.727461,2.735751
std,9.775097,13.104775,1.580354
min,8.000000,0.000000,1.000000
25%,17.000000,0.000000,1.000000
50%,19.000000,0.100000,2.000000
75%,22.000000,0.600000,4.000000
max,99.000000,150.000000,5.000000


A statistical and structural summary of the data was performed to gain a better understanding of the dataframe we are working with. The results demonstrated that “age” had a mean of 21.3 years (SD=9.7), “playtime” had a mean of 5.8 hours (SD=28.3), and “experience level” had a mean of 2.7 (SD=1.6) when rounded to one decimal place. 

In [7]:
players_filtered = players_filtered.assign(
    playtime_grouped=pd.cut(
        players_filtered["played_hours"],
        bins=[0, 1, 5, 10, 15, 20, 50, 100, 200, 223.1], 
        labels=["<1", "1–5", "5–10", "10–15", "15-20", "20–50", "50–100", "100-200", "200>"]
    )
)

players_plot = alt.Chart(players_filtered, title = "Playtime in Hours: Player age and Experience Level").mark_circle(size=40).encode(
    x=alt.X("age:Q").title("Age (years)"),
    y=alt.Y("experience_num:O", 
            title = "Experience (numbered low to high)",
            scale=alt.Scale(reverse=True)
), color=alt.Color("playtime_grouped:N", sort = ["<1", "1–5", "5–10", "10–15", "15-20", "20–50", "50–100", "100-200", "200>"]).scale(scheme="paired").title("Playtime (hours, grouped)")
).properties(width=850, height=200)
players_plot

alt.Chart(...)

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline


# Standardize age and experience_num 
players_preprocessor = make_column_transformer(
    (StandardScaler(), ["age", "experience_num"]),
    remainder="drop"
)

players_pipeline = make_pipeline(
    players_preprocessor,
    KNeighborsRegressor()
)


In [10]:
# GRID SEARCH (1–75 neighbors, step = 3) with 5-fold CV

param_grid = {
    "kneighborsregressor__n_neighbors": range(1, 76, 3)
}

players_gridsearch = GridSearchCV(
    estimator=players_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    return_train_score=False
)
print(players_gridsearch)
# Fit on training data

players_gridsearch.fit(
    training_data[["age", "experience_num"]],   
    training_data["played_hours"]              
)
print(players_gridsearch)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('standardscaler',
                                                                         StandardScaler(),
                                                                         ['age',
                                                                          'experience_num'])])),
                                       ('kneighborsregressor',
                                        KNeighborsRegressor())]),
             param_grid={'kneighborsregressor__n_neighbors': range(1, 76, 3)},
             scoring='neg_root_mean_squared_error')
GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('standardscaler',
                                                                         StandardScaler(),
                                      

/opt/conda/lib/python3.11/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [11]:
# Gather CV results 

cv_results = pd.DataFrame(players_gridsearch.cv_results_)

cv_results["sem_test_score"] = cv_results["std_test_score"] / (5**0.5)

cv_results = (
    cv_results[
        ["param_kneighborsregressor__n_neighbors",
         "mean_test_score",
         "sem_test_score"]
    ]
    .rename(columns={
        "param_kneighborsregressor__n_neighbors": "n_neighbors"
    })
)

cv_results["mean_test_score"] = -cv_results["mean_test_score"]

cv_results

,n_neighbors,mean_test_score,sem_test_score
0,1,28.452765,6.107046
1,4,27.058662,5.228597
2,7,22.474854,4.616850
3,10,23.987155,4.516612
4,13,23.673930,4.750189
5,16,22.515230,5.272415
6,19,22.360799,5.416053
7,22,22.070326,5.623141
8,25,21.705234,5.802995
9,28,21.739077,5.856564


In [12]:
players_gridsearch.best_params_

{'kneighborsregressor__n_neighbors': 73}

In [13]:
# Make prediction
testing_data["predicted"] = players_gridsearch.predict(
    testing_data[["age", "experience_num"]]
)

# Compute RMSPE
RMSPE = mean_squared_error(
    y_true=testing_data["played_hours"],
    y_pred=testing_data["predicted"]
)**0.5

RMSPE

np.float64(34.4122421618046)

In [14]:
#linear regression
X_train = training_data[["age", "experience_num"]]
y_train = training_data["played_hours"]
X_test = testing_data[["age", "experience_num"]]
y_test = testing_data["played_hours"]

lm = LinearRegression()
players_train_fit = lm.fit(X_train, y_train)
print(players_train_fit.coef_)
print(players_train_fit.intercept_)

[-0.08066138 -0.14859094]
6.937005449675615


In [15]:
#finding rmse
train_predictions = players_train_fit.predict(X_train)
lm_rmse = mean_squared_error(train_predictions, y_train)**0.5
print(lm_rmse)

25.137087062308986


In [16]:
#finding rmspe
test_predictions = players_train_fit.predict(X_test)
lm_rmspe = mean_squared_error(test_predictions, y_test)**0.5
print(lm_rmspe)

34.45033724343876


The RMSPE for the linear regression is lower than that of the KNN regression, making it a better option as a model.

**Multivairable linear regression**

In [17]:
from sklearn import linear_model

mvr_X = players_filtered[["age", "experience_num"]]
mvr_y = players_filtered["played_hours"]

mvr_reg = linear_model.LinearRegression()
mvr_reg.fit(mvr_X, mvr_y)

LinearRegression()

In [18]:
# Finding the coefficient: a factor that describes the relationship with an unknown variable.
print(mvr_reg.coef_)

[-0.14368895 -0.81510904]


The coefficient value for:
* "age": -0.14368895
* "experience": -0.81510904

**Pairplot linear regression**

In [26]:
#age_prediction_grid = players_filtered[["age"]].agg(["min", "max"])
#age_prediction_grid["predicted"] = lm.predict(age_prediction_grid)

age_scatter1 = alt.Chart(players_filtered, title="Player age and playtime").mark_point(opacity=0.5).encode(
    x=alt.X("age").scale(zero=False).title("Age (years)"),
    y=alt.Y("played_hours").title("Playtime (hours)")
).properties(width=200, height=400)

experience_scatter1 = alt.Chart(players_filtered, title="Player experience level and playtime").mark_point(opacity=0.5).encode(
    x=alt.X("experience_num:O").scale(zero=False).title("Experience level (lowest to highest)"),
    y=alt.Y("played_hours").title("Playtime (hours)")
).properties(width=200, height=400)

age_exp_scatter = alt.Chart(players_filtered, title="Player age and experience level").mark_point(opacity=0.5).encode(
    x=alt.X("experience_num:O").scale(zero=False).title("Experience level (lowest to highest)"),
    y=alt.Y("age").title("Age (years)")
).properties(width=200, height=400)

display(experience_scatter1 | age_scatter1 | age_exp_scatter)

alt.HConcatChart(...)

In [36]:
players_filtered_1 = players_filtered[(players_filtered["played_hours"] <= 5) & (players_filtered["age"] <=60)]
players_filtered_1

age_scatter2 = alt.Chart(players_filtered_1, title="Player age and playtime, excluding outliers").mark_point(opacity=0.5).encode(
    x=alt.X("age").scale(zero=False).title("Age (years)"),
    y=alt.Y("played_hours").title("Playtime (hours)")
).properties(width=250, height=500)


experience_scatter2 = alt.Chart(players_filtered_1, title="Player experience level and playtime, excluding outliers").mark_point(opacity=0.5).encode(
    x=alt.X("experience_num:O").scale(zero=False).title("Experience level (lowest to highest)"),
    y=alt.Y("played_hours").title("Playtime (hours)")
).properties(width=250, height=500)

age_exp_scatter = alt.Chart(players_filtered_1, title="Player age and experience level, excluding outliers").mark_point(opacity=0.5).encode(
    x=alt.X("experience_num:O").scale(zero=False).title("Experience level (lowest to highest)"),
    y=alt.Y("age").title("Age (years)")
).properties(width=250, height=500)

display(experience_scatter2 | age_scatter2 | age_exp_scatter)

alt.HConcatChart(...)

**Pairplot (delete?)**

In [23]:
# import altair as alt

# columns = ["age", "played_hours", "experience_num"]  # pick your numeric columns

# pair_plot = alt.Chart(players_filtered).mark_point().encode(
#     alt.X(alt.repeat("column"), type="quantitative"),
#     alt.Y(alt.repeat("row"), type="quantitative")
# ).properties(
#     width=250,
#     height=200
# ).repeat(
#     row=columns,
#     column=columns
# )

# pair_plot